## Data Cleaning 

Include physiochemcial properties

In [27]:
#import libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None )

## Yield Data

In [28]:
#import yield data
yld = pd.read_csv('../data/SAP2020_merged_v2.csv')
yld.head(3)

,PlotIDYellow,PlotIDRed,SorghumAccessionYellow,SorghumAccessionRed,SorghumNameRed,SNPDataID,Row,Column,Block,Treatment,DaysToBloom,MedianLeafAngle,LeafAngleSDV,PoorStand?,PaniclesPerPlot,PanicleGrainWeight,EstimatedPlotYield,FlagLeafLength,FlagLeafWidth,ExtantLeafNumber,PlantHeight,ThirdLeafLength,ThirdLeafWidth,TillersPerPlant,StemDiameterLower,StemDiameterUpper,RachisLength,RachisDiameterLower,RachisDiameterUpper,PrimaryBranchNo,BranchInternodeLength
0,9844,8390.0,PI 655998,PI 655998,KS19,294.0,69,1.0,8,SufficientNitrogen,62.0,43.2,6.29,N,15.0,20.0,300.0,50.1,4.5,11.0,86.0,68.0,6.0,0.00,21.01,9.92,27.37,6.31,1.78,81.00,1.80
1,8390,8377.0,PI 564163,BTx623,BTx623,160.0,69,2.0,8,SufficientNitrogen,65.0,43.7,7.11,Y,11.0,NaN,NaN,34.0,5.3,12.0,102.0,72.0,8.0,0.67,25.89,11.19,27.10,6.96,2.66,60.00,1.87
2,8377,8364.0,PI 576396,PI 576396,SC_324,196.0,69,3.0,8,SufficientNitrogen,64.0,45.4,7.80,N,12.0,38.0,456.0,24.2,3.6,12.0,90.0,60.0,5.0,0.00,16.02,7.30,9.63,5.42,4.61,35.33,1.77


In [29]:
#select only PlotID, SorghumAccessing Treatment and PanickleGrainWeight column
yld = yld [['PlotIDRed','PlotIDYellow','SorghumAccessionRed','SorghumAccessionYellow','Treatment',
            'PanicleGrainWeight','DaysToBloom','PlantHeight']]

In [30]:
#rename Sorghumacessing to PI and PanicleGrainWeight to yield
yld.rename(columns = {"SorghumAccessionRed":"PIRed",
                      "SorghumAccessionYellow":"PIYellow",
                      "PanicleGrainWeight":"yield",
                      "PlotIDRed":"PlotIDRed",
                     "PlotIDYellow":"PlotIDYellow"},inplace=True)

In [31]:
#change treatments labels
#LowNitrogen => LN
#SufficientNitrogen => SN

yld['Treatment'] = yld.Treatment.apply(lambda x: "HN" if x =="SufficientNitrogen" else "LN")

In [32]:
#drop nan
yld = yld[~yld.PlotIDRed.isnull()]

In [33]:
#round PlotID to nearest integer
import math
a = float('nan')
yld['PlotIDRed'] = yld.PlotIDRed.apply(lambda x: int(x))

In [34]:
#fill-in missing values with 0 on the yield data
#yld['yield'].fillna(0,inplace = True) #dropna 
yld = yld[~yld['DaysToBloom'].isnull()] 
yld.Treatment.value_counts()

HN    1073
LN     684
Name: Treatment, dtype: int64

In [35]:
#check NAN
yld.isnull().sum()

PlotIDRed         0
PlotIDYellow      0
PIRed             0
PIYellow          3
Treatment         0
yield           255
DaysToBloom       0
PlantHeight     381
dtype: int64

In [36]:
#filter key columns and rename them appropriately
d2b = yld [['PlotIDYellow','PIYellow','Treatment','DaysToBloom']]
d2b.rename(columns = {"PlotIDYellow":"PlotID","PIYellow":"PI"}, inplace = True)
print(d2b.shape)
d2b.head(3)

(1757, 4)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,PlotID,PI,Treatment,DaysToBloom
0,9844,PI 655998,HN,62.0
1,8390,PI 564163,HN,65.0
2,8377,PI 576396,HN,64.0


## Reflectance Data

In [37]:
#import yield data
spectra = pd.read_csv('../data/Sorghum_Hyperspectral_Data.csv')
spectra.head(3)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


crop PlotID  year         trt pheno      CHL       EWT       LWC  \
0  sorghum    281  2018  greenhouse   yes  446.944  0.021140  0.801818   
1  sorghum    282  2018  greenhouse   yes  713.800  0.013651  0.718788   
2  sorghum    283  2018  greenhouse   yes  724.911  0.013870  0.715429   

          SLA     N     P     K    Mg    Ca     S     Fe     Mn     B    Cu  \
0  191.380472  2.84  0.49  2.69  0.20  0.15  0.22  133.0   27.0  12.0  13.0   
1  187.244828  3.44  0.53  2.08  0.32  1.15  0.21  109.0   98.0  38.0   9.0   
2  181.260442  3.52  0.56  1.69  0.35  1.27  0.21  113.0  147.0  36.0   9.0   

     Zn      X350      X351      X352      X353      X354      X355      X356  \
0  37.0  0.142908  0.147872  0.131849  0.096525  0.058579  0.073648  0.090709   
1  64.0  0.144607  0.133411  0.127014  0.108473  0.062686  0.078634  0.097572   
2  62.0  0.133118  0.133659  0.129053  0.105945  0.059058  0.078518  0.098973   

       X357      X358      X359      X360      X361      X362      X363  \
0  0.091497  0.089564  0.073065  0.068222  0.080974  0.084464  0.087580   
1  0.098717  0.099297  0.075012  0.071114  0.095011  0.091853  0.087312   
2  0.097981  0.095278  0.070518  0.067381  0.092476  0.092793  0.086446   

       X364      X365      X366      X367      X368      X369      X370  \
0  0.080229  0.064086  0.083023  0.091025  0.085190  0.079682  0.092441   
1  0.077852  0.064859  0.081847  0.086992  0.082989  0.085961  0.098889   
2  0.076895  0.069685  0.087265  0.089406  0.083186  0.087562  0.096989   

       X371      X372      X373      X374      X375      X376      X377  \
0  0.086188  0.065738  0.063204  0.066281  0.073109  0.077155  0.065119   
1  0.090233  0.069327  0.070434  0.075417  0.078228  0.076858  0.072080   
2  0.086667  0.066244  0.066209  0.071976  0.076505  0.076258  0.070422   

       X378      X379      X380      X381      X382      X383      X384  \
0  0.068713  0.074253  0.069952  0.071280  0.067743  0.064909  0.069064   
1  0.076396  0.078304  0.071591  0.074719  0.073626  0.070162  0.069692   
2  0.073642  0.075110  0.068252  0.069625  0.065444  0.062526  0.068870   

       X385      X386      X387      X388      X389      X390      X391  \
0  0.062337  0.063613  0.070386  0.066614  0.067137  0.071195  0.074272   
1  0.065991  0.068167  0.073365  0.071653  0.072162  0.074712  0.076154   
2  0.062721  0.064474  0.072066  0.068888  0.068324  0.070374  0.072274   

       X392      X393      X394      X395      X396      X397      X398  \
0  0.069974  0.068177  0.067990  0.067076  0.065771  0.069722  0.073364   
1  0.071298  0.071318  0.071446  0.067839  0.068895  0.072668  0.075067   
2  0.070088  0.067754  0.066173  0.065423  0.065095  0.069236  0.072735   

       X399      X400      X401      X402      X403      X404      X405  \
0  0.069445  0.065306  0.065402  0.068221  0.068122  0.069550  0.070833   
1  0.072547  0.067586  0.067888  0.071296  0.068660  0.069635  0.072082   
2  0.069307  0.065058  0.064915  0.067179  0.065646  0.066940  0.069100   

       X406      X407      X408      X409      X410      X411      X412  \
0  0.070859  0.070629  0.071852  0.072408  0.070876  0.071275  0.071306   
1  0.072835  0.069704  0.070873  0.073089  0.072369  0.072776  0.073111   
2  0.069834  0.068238  0.068724  0.069210  0.067783  0.068870  0.069666   

       X413      X414      X415      X416      X417      X418      X419  \
0  0.070232  0.068640  0.069769  0.070692  0.070718  0.071919  0.073305   
1  0.071973  0.068971  0.070703  0.071893  0.070910  0.070876  0.072144   
2  0.068674  0.066111  0.068287  0.069397  0.068036  0.068288  0.070305   

       X420      X421      X422      X423      X424      X425      X426  \
0  0.072214  0.068908  0.070574  0.072576  0.072232  0.069378  0.072004   
1  0.071891  0.069606  0.071006  0.073501  0.073903  0.070782  0.071337   
2  0.070104  0.066760  0.067481  0.069210  0.069176  0.066448  0.068050   

       X427      X428      X429      X430      X431      

In [38]:
#mport metadata for spectra data
meta = pd.read_csv('../data/Sorghum_Field_Map_2020.csv')
meta.head(3)

,PlotID,Asseccion,Block,Trt
0,5001,PI 656057,5,HN
1,5002,PI 656011,5,HN
2,5003,PI 576347,5,HN


In [39]:
#change plotID to object 
meta.PlotID = meta.PlotID.astype('object')

In [40]:
#join spectra data and metadata
ref = spectra.merge(meta,left_on = ['PlotID','trt'], right_on  = ['PlotID','Trt'],how = 'left')
ref.head(3)

crop PlotID  year         trt pheno      CHL       EWT       LWC  \
0  sorghum    281  2018  greenhouse   yes  446.944  0.021140  0.801818   
1  sorghum    282  2018  greenhouse   yes  713.800  0.013651  0.718788   
2  sorghum    283  2018  greenhouse   yes  724.911  0.013870  0.715429   

          SLA     N     P     K    Mg    Ca     S     Fe     Mn     B    Cu  \
0  191.380472  2.84  0.49  2.69  0.20  0.15  0.22  133.0   27.0  12.0  13.0   
1  187.244828  3.44  0.53  2.08  0.32  1.15  0.21  109.0   98.0  38.0   9.0   
2  181.260442  3.52  0.56  1.69  0.35  1.27  0.21  113.0  147.0  36.0   9.0   

     Zn      X350      X351      X352      X353      X354      X355      X356  \
0  37.0  0.142908  0.147872  0.131849  0.096525  0.058579  0.073648  0.090709   
1  64.0  0.144607  0.133411  0.127014  0.108473  0.062686  0.078634  0.097572   
2  62.0  0.133118  0.133659  0.129053  0.105945  0.059058  0.078518  0.098973   

       X357      X358      X359      X360      X361      X362      X363  \
0  0.091497  0.089564  0.073065  0.068222  0.080974  0.084464  0.087580   
1  0.098717  0.099297  0.075012  0.071114  0.095011  0.091853  0.087312   
2  0.097981  0.095278  0.070518  0.067381  0.092476  0.092793  0.086446   

       X364      X365      X366      X367      X368      X369      X370  \
0  0.080229  0.064086  0.083023  0.091025  0.085190  0.079682  0.092441   
1  0.077852  0.064859  0.081847  0.086992  0.082989  0.085961  0.098889   
2  0.076895  0.069685  0.087265  0.089406  0.083186  0.087562  0.096989   

       X371      X372      X373      X374      X375      X376      X377  \
0  0.086188  0.065738  0.063204  0.066281  0.073109  0.077155  0.065119   
1  0.090233  0.069327  0.070434  0.075417  0.078228  0.076858  0.072080   
2  0.086667  0.066244  0.066209  0.071976  0.076505  0.076258  0.070422   

       X378      X379      X380      X381      X382      X383      X384  \
0  0.068713  0.074253  0.069952  0.071280  0.067743  0.064909  0.069064   
1  0.076396  0.078304  0.071591  0.074719  0.073626  0.070162  0.069692   
2  0.073642  0.075110  0.068252  0.069625  0.065444  0.062526  0.068870   

       X385      X386      X387      X388      X389      X390      X391  \
0  0.062337  0.063613  0.070386  0.066614  0.067137  0.071195  0.074272   
1  0.065991  0.068167  0.073365  0.071653  0.072162  0.074712  0.076154   
2  0.062721  0.064474  0.072066  0.068888  0.068324  0.070374  0.072274   

       X392      X393      X394      X395      X396      X397      X398  \
0  0.069974  0.068177  0.067990  0.067076  0.065771  0.069722  0.073364   
1  0.071298  0.071318  0.071446  0.067839  0.068895  0.072668  0.075067   
2  0.070088  0.067754  0.066173  0.065423  0.065095  0.069236  0.072735   

       X399      X400      X401      X402      X403      X404      X405  \
0  0.069445  0.065306  0.065402  0.068221  0.068122  0.069550  0.070833   
1  0.072547  0.067586  0.067888  0.071296  0.068660  0.069635  0.072082   
2  0.069307  0.065058  0.064915  0.067179  0.065646  0.066940  0.069100   

       X406      X407      X408      X409      X410      X411      X412  \
0  0.070859  0.070629  0.071852  0.072408  0.070876  0.071275  0.071306   
1  0.072835  0.069704  0.070873  0.073089  0.072369  0.072776  0.073111   
2  0.069834  0.068238  0.068724  0.069210  0.067783  0.068870  0.069666   

       X413      X414      X415      X416      X417      X418      X419  \
0  0.070232  0.068640  0.069769  0.070692  0.070718  0.071919  0.073305   
1  0.071973  0.068971  0.070703  0.071893  0.070910  0.070876  0.072144   
2  0.068674  0.066111  0.068287  0.069397  0.068036  0.068288  0.070305   

       X420      X421      X422      X423      X424      X425      X426  \
0  0.072214  0.068908  0.070574  0.072576  0.072232  0.069378  0.072004   
1  0.071891  0.069606  0.071006  0.073501  0.073903  0.070782  0.071337   
2  0.070104  0.066760  0.067481  0.069210  0.069176  0.066448  0.068050   

       X427      X428      X429      X430      X431      

In [41]:
#wavelengths
pchem = list(ref.columns)[5:20]
waves = [x for x in list(ref.columns) if x.startswith('X')]
ref = ref[['PlotID','trt','Asseccion']+pchem +waves]
ref.head(3)

PlotID         trt Asseccion      CHL       EWT       LWC         SLA     N  \
0    281  greenhouse       NaN  446.944  0.021140  0.801818  191.380472  2.84   
1    282  greenhouse       NaN  713.800  0.013651  0.718788  187.244828  3.44   
2    283  greenhouse       NaN  724.911  0.013870  0.715429  181.260442  3.52   

      P     K    Mg    Ca     S     Fe     Mn     B    Cu    Zn      X350  \
0  0.49  2.69  0.20  0.15  0.22  133.0   27.0  12.0  13.0  37.0  0.142908   
1  0.53  2.08  0.32  1.15  0.21  109.0   98.0  38.0   9.0  64.0  0.144607   
2  0.56  1.69  0.35  1.27  0.21  113.0  147.0  36.0   9.0  62.0  0.133118   

       X351      X352      X353      X354      X355      X356      X357  \
0  0.147872  0.131849  0.096525  0.058579  0.073648  0.090709  0.091497   
1  0.133411  0.127014  0.108473  0.062686  0.078634  0.097572  0.098717   
2  0.133659  0.129053  0.105945  0.059058  0.078518  0.098973  0.097981   

       X358      X359      X360      X361      X362      X363      X364  \
0  0.089564  0.073065  0.068222  0.080974  0.084464  0.087580  0.080229   
1  0.099297  0.075012  0.071114  0.095011  0.091853  0.087312  0.077852   
2  0.095278  0.070518  0.067381  0.092476  0.092793  0.086446  0.076895   

       X365      X366      X367      X368      X369      X370      X371  \
0  0.064086  0.083023  0.091025  0.085190  0.079682  0.092441  0.086188   
1  0.064859  0.081847  0.086992  0.082989  0.085961  0.098889  0.090233   
2  0.069685  0.087265  0.089406  0.083186  0.087562  0.096989  0.086667   

       X372      X373      X374      X375      X376      X377      X378  \
0  0.065738  0.063204  0.066281  0.073109  0.077155  0.065119  0.068713   
1  0.069327  0.070434  0.075417  0.078228  0.076858  0.072080  0.076396   
2  0.066244  0.066209  0.071976  0.076505  0.076258  0.070422  0.073642   

       X379      X380      X381      X382      X383      X384      X385  \
0  0.074253  0.069952  0.071280  0.067743  0.064909  0.069064  0.062337   
1  0.078304  0.071591  0.074719  0.073626  0.070162  0.069692  0.065991   
2  0.075110  0.068252  0.069625  0.065444  0.062526  0.068870  0.062721   

       X386      X387      X388      X389      X390      X391      X392  \
0  0.063613  0.070386  0.066614  0.067137  0.071195  0.074272  0.069974   
1  0.068167  0.073365  0.071653  0.072162  0.074712  0.076154  0.071298   
2  0.064474  0.072066  0.068888  0.068324  0.070374  0.072274  0.070088   

       X393      X394      X395      X396      X397      X398      X399  \
0  0.068177  0.067990  0.067076  0.065771  0.069722  0.073364  0.069445   
1  0.071318  0.071446  0.067839  0.068895  0.072668  0.075067  0.072547   
2  0.067754  0.066173  0.065423  0.065095  0.069236  0.072735  0.069307   

       X400      X401      X402      X403      X404      X405      X406  \
0  0.065306  0.065402  0.068221  0.068122  0.069550  0.070833  0.070859   
1  0.067586  0.067888  0.071296  0.068660  0.069635  0.072082  0.072835   
2  0.065058  0.064915  0.067179  0.065646  0.066940  0.069100  0.069834   

       X407      X408      X409      X410      X411      X412      X413  \
0  0.070629  0.071852  0.072408  0.070876  0.071275  0.071306  0.070232   
1  0.069704  0.070873  0.073089  0.072369  0.072776  0.073111  0.071973   
2  0.068238  0.068724  0.069210  0.067783  0.068870  0.069666  0.068674   

       X414      X415      X416      X417      X418      X419      X420  \
0  0.068640  0.069769  0.070692  0.070718  0.071919  0.073305  0.072214   
1  0.068971  0.070703  0.071893  0.070910  0.070876  0.072144  0.071891   
2  0.066111  0.068287  0.069397  0.068036  0.068288  0.070305  0.070104   

       X421      X422      X423      X424      X425      X426      X427  \
0  0.068908  0.070574  0.072576  0.072232  0.069378  0.072004  0.072171   
1  0.069606  0.071006  0.073501  0.073903  0.070782  0.071337  0.071419   
2  0.066760  0.067481  0.069210  0.069176  0.066448  0.068050  0.068442   

       X428      X429      X430      X431      X432      

In [42]:
#rename trt to Treatment
ref.rename(columns={'trt':"Treatment",'Asseccion':'PI'},inplace = True)
ref.head(3)

PlotID   Treatment   PI      CHL       EWT       LWC         SLA     N  \
0    281  greenhouse  NaN  446.944  0.021140  0.801818  191.380472  2.84   
1    282  greenhouse  NaN  713.800  0.013651  0.718788  187.244828  3.44   
2    283  greenhouse  NaN  724.911  0.013870  0.715429  181.260442  3.52   

      P     K    Mg    Ca     S     Fe     Mn     B    Cu    Zn      X350  \
0  0.49  2.69  0.20  0.15  0.22  133.0   27.0  12.0  13.0  37.0  0.142908   
1  0.53  2.08  0.32  1.15  0.21  109.0   98.0  38.0   9.0  64.0  0.144607   
2  0.56  1.69  0.35  1.27  0.21  113.0  147.0  36.0   9.0  62.0  0.133118   

       X351      X352      X353      X354      X355      X356      X357  \
0  0.147872  0.131849  0.096525  0.058579  0.073648  0.090709  0.091497   
1  0.133411  0.127014  0.108473  0.062686  0.078634  0.097572  0.098717   
2  0.133659  0.129053  0.105945  0.059058  0.078518  0.098973  0.097981   

       X358      X359      X360      X361      X362      X363      X364  \
0  0.089564  0.073065  0.068222  0.080974  0.084464  0.087580  0.080229   
1  0.099297  0.075012  0.071114  0.095011  0.091853  0.087312  0.077852   
2  0.095278  0.070518  0.067381  0.092476  0.092793  0.086446  0.076895   

       X365      X366      X367      X368      X369      X370      X371  \
0  0.064086  0.083023  0.091025  0.085190  0.079682  0.092441  0.086188   
1  0.064859  0.081847  0.086992  0.082989  0.085961  0.098889  0.090233   
2  0.069685  0.087265  0.089406  0.083186  0.087562  0.096989  0.086667   

       X372      X373      X374      X375      X376      X377      X378  \
0  0.065738  0.063204  0.066281  0.073109  0.077155  0.065119  0.068713   
1  0.069327  0.070434  0.075417  0.078228  0.076858  0.072080  0.076396   
2  0.066244  0.066209  0.071976  0.076505  0.076258  0.070422  0.073642   

       X379      X380      X381      X382      X383      X384      X385  \
0  0.074253  0.069952  0.071280  0.067743  0.064909  0.069064  0.062337   
1  0.078304  0.071591  0.074719  0.073626  0.070162  0.069692  0.065991   
2  0.075110  0.068252  0.069625  0.065444  0.062526  0.068870  0.062721   

       X386      X387      X388      X389      X390      X391      X392  \
0  0.063613  0.070386  0.066614  0.067137  0.071195  0.074272  0.069974   
1  0.068167  0.073365  0.071653  0.072162  0.074712  0.076154  0.071298   
2  0.064474  0.072066  0.068888  0.068324  0.070374  0.072274  0.070088   

       X393      X394      X395      X396      X397      X398      X399  \
0  0.068177  0.067990  0.067076  0.065771  0.069722  0.073364  0.069445   
1  0.071318  0.071446  0.067839  0.068895  0.072668  0.075067  0.072547   
2  0.067754  0.066173  0.065423  0.065095  0.069236  0.072735  0.069307   

       X400      X401      X402      X403      X404      X405      X406  \
0  0.065306  0.065402  0.068221  0.068122  0.069550  0.070833  0.070859   
1  0.067586  0.067888  0.071296  0.068660  0.069635  0.072082  0.072835   
2  0.065058  0.064915  0.067179  0.065646  0.066940  0.069100  0.069834   

       X407      X408      X409      X410      X411      X412      X413  \
0  0.070629  0.071852  0.072408  0.070876  0.071275  0.071306  0.070232   
1  0.069704  0.070873  0.073089  0.072369  0.072776  0.073111  0.071973   
2  0.068238  0.068724  0.069210  0.067783  0.068870  0.069666  0.068674   

       X414      X415      X416      X417      X418      X419      X420  \
0  0.068640  0.069769  0.070692  0.070718  0.071919  0.073305  0.072214   
1  0.068971  0.070703  0.071893  0.070910  0.070876  0.072144  0.071891   
2  0.066111  0.068287  0.069397  0.068036  0.068288  0.070305  0.070104   

       X421      X422      X423      X424      X425      X426      X427  \
0  0.068908  0.070574  0.072576  0.072232  0.069378  0.072004  0.072171   
1  0.069606  0.071006  0.073501  0.073903  0.070782  0.071337  0.071419   
2  0.066760  0.067481  0.069210  0.069176  0.066448  0.068050  0.068442   

       X428      X429      X430      X431      X432      X433      X434  \
0 

## Join Yield Data and Reflectance Data

In [43]:
print(d2b.shape)
d2b.head(3)

(1757, 4)


,PlotID,PI,Treatment,DaysToBloom
0,9844,PI 655998,HN,62.0
1,8390,PI 564163,HN,65.0
2,8377,PI 576396,HN,64.0


In [49]:
#Inner join of yield data and reflectance data based on PlotID, Treatment and PI

#merge on the yellow bands
#df_yellow = ref.merge(d2b,on=['PlotID','PI','Treatment'], how = 'inner')
df_yellow = ref.merge(d2b[['PlotID','DaysToBloom']],on='PlotID', how = 'inner')
print(df_yellow.shape)
print(df_yellow.Treatment.value_counts())
df_yellow.head(3)

(834, 2170)
LN    533
HN    301
Name: Treatment, dtype: int64


PlotID Treatment         PI       CHL       EWT       LWC        SLA      N  \
0   5001        HN  PI 656057  467.6750  0.048621  0.816438  91.477861  3.612   
1   5002        HN  PI 656011  512.3625  0.043236  0.768144  76.626487  2.916   
2   5003        HN  PI 576347  606.6625  0.033848  0.751805  89.492182  3.749   

       P     K     Mg     Ca      S     Fe    Mn    B    Cu    Zn      X350  \
0  0.381  1.93  0.245  0.674  0.206  171.0  31.0  6.4  17.6  28.0  0.085242   
1  0.387  2.24  0.143  0.447  0.196  134.0  34.0  4.6  12.5  27.0  0.053529   
2  0.492  1.54  0.260  0.583  0.196  159.0  36.0  5.1  15.4  32.0  0.133649   

       X351      X352      X353      X354      X355      X356      X357  \
0  0.084198  0.073049  0.062852  0.071412  0.076959  0.086402  0.089655   
1  0.087761  0.089831  0.066536  0.070464  0.065440  0.057905  0.061879   
2  0.113164  0.095398  0.095292  0.097985  0.089531  0.085457  0.086694   

       X358      X359      X360      X361      X362      X363      X364  \
0  0.068680  0.077627  0.075629  0.049378  0.047664  0.048732  0.053430   
1  0.087529  0.083688  0.066741  0.054267  0.042131  0.050978  0.056994   
2  0.082119  0.090674  0.082183  0.057630  0.074017  0.063699  0.056483   

       X365      X366      X367      X368      X369      X370      X371  \
0  0.059725  0.043177  0.055788  0.069612  0.049814  0.054452  0.055179   
1  0.045107  0.046368  0.043490  0.042569  0.050927  0.046258  0.041853   
2  0.072555  0.051468  0.052361  0.063759  0.055830  0.039474  0.044329   

       X372      X373      X374      X375      X376      X377      X378  \
0  0.045720  0.038621  0.047542  0.053812  0.049443  0.042255  0.043361   
1  0.043365  0.048262  0.047062  0.048264  0.051650  0.044692  0.043194   
2  0.059886  0.055076  0.055611  0.054215  0.045810  0.032091  0.028689   

       X379      X380      X381      X382      X383      X384      X385  \
0  0.048633  0.051029  0.044424  0.043408  0.041854  0.032599  0.041451   
1  0.040966  0.035022  0.041995  0.044352  0.042166  0.041462  0.043841   
2  0.033169  0.038722  0.042077  0.040161  0.037917  0.041208  0.046213   

       X386      X387      X388      X389      X390      X391      X392  \
0  0.048299  0.048850  0.050056  0.046709  0.043648  0.042465  0.038687   
1  0.043980  0.042063  0.041588  0.038251  0.035009  0.034469  0.037531   
2  0.048145  0.046461  0.043884  0.043052  0.039607  0.035373  0.041091   

       X393      X394      X395      X396      X397      X398      X399  \
0  0.046113  0.051060  0.044858  0.047175  0.049994  0.049110  0.043823   
1  0.040736  0.042420  0.042195  0.041240  0.040235  0.039823  0.040379   
2  0.045182  0.045669  0.043754  0.042794  0.041254  0.040067  0.040752   

       X400      X401      X402      X403      X404      X405      X406  \
0  0.043862  0.041952  0.039572  0.044624  0.044988  0.046128  0.048967   
1  0.041654  0.042781  0.042810  0.041331  0.044993  0.048847  0.048443   
2  0.043151  0.043261  0.042180  0.044544  0.044126  0.043205  0.043584   

       X407      X408      X409      X410      X411      X412      X413  \
0  0.043201  0.043568  0.048380  0.051787  0.049475  0.048316  0.046960   
1  0.042648  0.044995  0.048170  0.044979  0.047369  0.049684  0.049955   
2  0.044392  0.045137  0.045063  0.044390  0.046864  0.045013  0.043726   

       X414      X415      X416      X417      X418      X419      X420  \
0  0.041976  0.046816  0.049948  0.048506  0.050128  0.048849  0.048119   
1  0.048659  0.048421  0.048269  0.048416  0.050394  0.048258  0.048416   
2  0.050980  0.051151  0.049499  0.049354  0.050059  0.049096  0.048275   

       X421      X422      X423      X424      X425      X426      X427  \
0  0.050257  0.051968  0.050424  0.047950  0.047706  0.047331  0.047042   
1  0.052975  0.049087  0.050950  0.054511  0.052481  0.052212  0.052104   
2  0.048961  0.049973  0.049505  0.048687  0.049145  0.049398  0.051126   

       X428      X429      X430      X431      X4

In [50]:
#save dataframe
df_yellow.to_csv('../data/data-1.5-yellow.csv')

In [51]:
#filter key columns and rename them appropriately
d2b = yld [['PlotIDRed','PIRed','Treatment','DaysToBloom']]
d2b.rename(columns = {"PlotIDRed":"PlotID","PIRed":"PI"}, inplace = True)
print(d2b.shape)
d2b.head(3)

(1757, 4)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,PlotID,PI,Treatment,DaysToBloom
0,8390,PI 655998,HN,62.0
1,8377,BTx623,HN,65.0
2,8364,PI 576396,HN,64.0


In [52]:
df_red = ref.merge(d2b[['PlotID','DaysToBloom']],on='PlotID', how = 'inner')
print(df_red.shape)
print(df_red.Treatment.value_counts())
df_red.head(3)

(827, 2170)
LN    532
HN    295
Name: Treatment, dtype: int64


PlotID Treatment   PI       CHL       EWT       LWC        SLA      N  \
0   5004        HN  NaN  468.6625  0.031408  0.748322  94.669000  3.761   
1   5005        HN  NaN  559.1375  0.031563  0.732255  86.647953  3.180   
2   5006        HN  NaN  568.8375  0.030689  0.716905  82.517814  3.161   

       P     K     Mg     Ca      S     Fe    Mn    B    Cu    Zn      X350  \
0  0.372  1.49  0.209  0.726  0.200  123.0  28.0  4.4  14.7  39.0  0.051370   
1  0.454  2.32  0.184  0.542  0.192  162.0  28.0  4.2  11.0  35.0  0.060103   
2  0.355  1.76  0.169  0.577  0.174  176.0  27.0  4.7  11.4  31.0  0.099085   

       X351      X352      X353      X354      X355      X356      X357  \
0  0.052746  0.045821  0.052313  0.083602  0.062361  0.049722  0.059869   
1  0.049632  0.045581  0.067715  0.117364  0.081665  0.061088  0.077107   
2  0.113754  0.114285  0.109921  0.113212  0.119975  0.100022  0.062165   

       X358      X359      X360      X361      X362      X363      X364  \
0  0.056525  0.061510  0.053292  0.032932  0.040317  0.062231  0.070485   
1  0.068005  0.072490  0.064835  0.043043  0.049886  0.063621  0.065962   
2  0.049947  0.069289  0.077344  0.060909  0.061961  0.067800  0.066358   

       X365      X366      X367      X368      X369      X370      X371  \
0  0.052649  0.041792  0.042909  0.043065  0.032088  0.044064  0.046019   
1  0.051330  0.043448  0.041333  0.039668  0.035503  0.045181  0.046671   
2  0.054756  0.053663  0.052614  0.051131  0.052293  0.058782  0.059387   

       X372      X373      X374      X375      X376      X377      X378  \
0  0.038522  0.043813  0.047412  0.047961  0.046872  0.047312  0.048542   
1  0.040810  0.043458  0.050485  0.052210  0.047122  0.044935  0.047376   
2  0.054000  0.050389  0.053716  0.050787  0.041124  0.042028  0.043616   

       X379      X380      X381      X382      X383      X384      X385  \
0  0.048327  0.045896  0.044430  0.043528  0.044992  0.049225  0.047608   
1  0.047966  0.043590  0.042818  0.040720  0.040671  0.045761  0.044807   
2  0.044856  0.045758  0.044086  0.046838  0.048902  0.044437  0.044277   

       X386      X387      X388      X389      X390      X391      X392  \
0  0.047901  0.048735  0.043370  0.042375  0.045063  0.047908  0.045674   
1  0.047601  0.050887  0.044123  0.043528  0.045290  0.044929  0.040516   
2  0.045808  0.046552  0.045058  0.043993  0.043806  0.042900  0.037509   

       X393      X394      X395      X396      X397      X398      X399  \
0  0.041191  0.040956  0.046317  0.043229  0.042814  0.045638  0.047879   
1  0.038967  0.041709  0.045823  0.040394  0.040797  0.045123  0.045981   
2  0.041162  0.045486  0.043184  0.042651  0.039952  0.037677  0.039974   

       X400      X401      X402      X403      X404      X405      X406  \
0  0.047047  0.044441  0.041939  0.041561  0.044752  0.046757  0.045050   
1  0.047368  0.047117  0.045181  0.043589  0.046269  0.047893  0.045604   
2  0.045524  0.046478  0.043937  0.045681  0.044032  0.041172  0.041084   

       X407      X408      X409      X410      X411      X412      X413  \
0  0.041494  0.043331  0.047099  0.048029  0.046650  0.048133  0.051203   
1  0.042084  0.044874  0.048860  0.048547  0.048192  0.049840  0.052071   
2  0.048657  0.051887  0.049826  0.046015  0.047787  0.052214  0.053085   

       X414      X415      X416      X417      X418      X419      X420  \
0  0.051696  0.050021  0.049206  0.050117  0.051099  0.050420  0.049987   
1  0.052230  0.052194  0.052084  0.052235  0.053210  0.051567  0.050060   
2  0.046039  0.049368  0.051306  0.048266  0.047596  0.049694  0.052982   

       X421      X422      X423      X424      X425      X426      X427  \
0  0.050363  0.046971  0.048846  0.051978  0.050416  0.050639  0.051851   
1  0.050250  0.048140  0.050010  0.052219  0.050339  0.052283  0.054480   
2  0.054911  0.051374  0.049807  0.050570  0.051625  0.052918  0.053335   

       X428      X429      X430      X431      X432      X433      X434  

In [53]:
df_red.to_csv('../data/data-1.5-red.csv')